# AADT Confidence Interval - State Route 99, District 3

## FHWA Links
* Guidelines for Obtaining AADT Estimates from Non-Traditional Sources:
    * https://www.fhwa.dot.gov/policyinformation/travel_monitoring/pubs/aadtnt/Guidelines_for_AADT_Estimates_Final.pdf
  
  
## AADT Analysis Locations
* 10 locations were used in the analysis
* Locations were determined based on the location on installed & recording Traffic Operations cameras
    * for additional information contact Zhenyu Zhu with Traffic Operations

## Traffic Census Data
* https://dot.ca.gov/programs/traffic-operations/census/traffic-volumes
* Back AADT, Peak Month, and Peak Hour usually represents traffic South or West of the count location.  
* Ahead AADT, Peak Month, and Peak Hour usually represents traffic North or East of the count location. Listing of routes with their designated  

* Because the Back & Ahead counts are included at each location in the Traffic Census Data, (e.g., "IRWINDALE, ARROW HIGHWAY") only one [OBJECTID*] per location was pulled; for this analysis the North Bound Nodes were used for the analysis. 
    * for more information see the diagram: https://traffic.onramp.dot.ca.gov/downloads/traffic/files/performance/census/Back_and_Ahead_Leg_Traffic_Count_Diagram.pdf

## StreetLight Analysis Data
* StreetLight Locations on Interstate 99 are one-direction, each location will contain two points: northbound and southbound
    * Analysis Type == Network Performance
    * Segment Metrics
    * 2022 was used to match currently available Traffic Census Data (as of 8/27/2025)
    * pulled a variety of Day Types, but plan to just look at """All Day Types"""
    * pulled a variety of Day Parts, but plan to just look at """All Day Parts"""




## import packages

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import csv

### Pull in the Location Dictionaries

In [2]:
# pull in the coordinates from the utils docs
#from osow_frp_o_d_utils_v3 import origin_intersections, destination_intersections
from shs_ct_tc_locations_utils import sr_99_d3_tc_aadt_locations

### Identify the Google Cloud Storage path

In [3]:
# Identify the GCS path to the data
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/compare_traffic_counts/sr99_d3/"

## Step 0, Pull in the Data

In [4]:
# This function will pull in the data and clean the column headers in a way that will make them easier to work with
def getdata_and_cleanheaders(path):
    # Read the CSV file
    df = pd.read_csv(path)

    # Clean column headers: remove spaces, convert to lowercase, and strip trailing asterisks
    cleaned_columns = []
    for column in df.columns:
        cleaned_column = column.replace(" ", "").lower().rstrip("*")
        cleaned_columns.append(cleaned_column)

    df.columns = cleaned_columns
    return df

In [5]:
# pull in the data & create dataframes
df_tc = getdata_and_cleanheaders(f"{gcs_path}caltrans_traffic_census_2022.csv")  # Traffic Census

In [6]:
# Identify the StreetLight Analysis to be used in the AADT comparison
df_stl = getdata_and_cleanheaders(f"{gcs_path}streetlight_sr99_d3_all_vehicles_2022_np.csv")  # StreetLight

In [7]:
# comparing
df_tc.to_csv("df_tc.csv", index=False)

In [8]:
# comparing
df_stl.to_csv("df_stl.csv", index=False)

## Step 1, Build a per-location summary of Traffic Census locations

In [9]:


def traditional_aadt_by_location(aadt_locations, df_tc, as_df=True):
    """
    Build a per-location summary of *traditional* (Traffic Census) AADT.
    Output columns:
      location, daytype, objectids, n_objectids, n_found_in_tc, missing_objectids,
      traditional_ahead_mean, traditional_behind_mean, traditional_aadt
    """

    def _ensure_list(x):
        if x is None: return []
        if isinstance(x, (list, tuple, set)): return list(x)
        return [x]

    def _gather_objectids(node_dict):
        ids = []
        if not isinstance(node_dict, dict): return ids
        if "objectid"  in node_dict: ids.extend(_ensure_list(node_dict["objectid"]))
        if "objectids" in node_dict: ids.extend(_ensure_list(node_dict["objectids"]))
        return [str(i) for i in ids if i is not None and str(i).strip() != ""]

    def _dedup(seq):
        seen=set(); out=[]
        for x in seq:
            if x not in seen:
                out.append(x); seen.add(x)
        return out

    def _normalize_one_location(name, loc):
        nodes = loc.get("nodes", {}) or {}
        all_ids=[]
        for _, node in nodes.items():
            all_ids.extend(_gather_objectids(node))
        return {
            "name": name,
            "daytype": loc.get("daytype", "0: All Days (M-Su)"),
            "objectids": _dedup(all_ids),
        }

    def _normalize_input(aadt_locs):
        if isinstance(aadt_locs, pd.DataFrame) and \
           {"name","daytype","objectids"}.issubset(aadt_locs.columns):
            return aadt_locs.to_dict(orient="records")
        if isinstance(aadt_locs, list) and aadt_locs and isinstance(aadt_locs[0], dict) and \
           {"name","daytype","objectids"}.issubset(aadt_locs[0].keys()):
            return aadt_locs

        recs = []
        if isinstance(aadt_locs, dict):
            for nm, loc in aadt_locs.items():
                recs.append(_normalize_one_location(nm, loc))
        elif isinstance(aadt_locs, list):
            for item in aadt_locs:
                if isinstance(item, dict) and "nodes" in item:  # single location dict
                    nm = item.get("location_description") or item.get("name") or "UNKNOWN"
                    recs.append(_normalize_one_location(nm, item))
                elif isinstance(item, dict):  # dict keyed by name
                    for nm, loc in item.items():
                        recs.append(_normalize_one_location(nm, loc))
        return recs

    def _traditional_aadt_for_ids(df_tc_in, obj_ids):
        if not obj_ids:
            return np.nan, np.nan, np.nan, 0
        sub = df_tc_in[df_tc_in["objectid"].astype(str).isin(obj_ids)]
        if sub.empty:
            return np.nan, np.nan, np.nan, 0
        ahead_vals = pd.to_numeric(sub.get("ahead_aadt"), errors="coerce").dropna()
        back_vals  = pd.to_numeric(sub.get("back_aadt"),  errors="coerce").dropna()
        mean_ahead = ahead_vals.mean() if not ahead_vals.empty else np.nan
        mean_back  = back_vals.mean()  if not back_vals.empty  else np.nan
        overall = np.nanmean([mean_ahead, mean_back])
        count_used = len(sub)
        return overall, mean_ahead, mean_back, count_used

    norm = _normalize_input(aadt_locations)
    tc_ids_all = set(df_tc["objectid"].astype(str).unique())

    rows = []
    for loc in norm:
        obj_ids = [str(x) for x in (loc.get("objectids") or [])]
        overall, mean_ahead, mean_back, n_found = _traditional_aadt_for_ids(df_tc, obj_ids)
        missing = [x for x in obj_ids if x not in tc_ids_all]

        # >>> CHANGE #1: pipe-join for display/export
        objectids_display = "|".join(obj_ids)

        rows.append({
            "location": loc.get("name"),
            "daytype":  loc.get("daytype"),
            "objectids": objectids_display,   # pipe-separated string
            "n_objectids": len(obj_ids),
            "n_found_in_tc": int(n_found),
            "missing_objectids": "|".join(missing) if missing else "",
            "traditional_ahead_mean": mean_ahead,
            "traditional_behind_mean": mean_back,
            "traditional_aadt": overall,
        })

    return pd.DataFrame(rows) if as_df else rows

In [10]:
# run step 1 - traditional aadt counts
trad_df = traditional_aadt_by_location(sr_99_d3_tc_aadt_locations, df_tc, as_df=True)

In [11]:
#trad_df.head()

In [12]:
# Export Step 1 as a CSV to take a look
trad_df.to_csv("step_1_traditional_aadt_by_location.csv", index=False)

## Step 2 Identify Traffic Census location names for the StreetLight segments

In [13]:
# Step 2
# Identify the Traffic Census Location name for the StreetLight segments
    # Traffic Census locations can either be bi-directional or uni-directional. 
        # If they are bi-directional they will only have one node per location, 
        # if they are uni-directional there will be two nodes per location. 
    # Each Traffic Census location has one an [AHEAD_AADT] and one [BEHIND_AADT] count per location node - so each Traffic Census location also has at least two StreetLight locations, 
        # but each location typically has four StreetLight segments (two Traffic Census nodes (e.g., Northbound & Southbound, each node has two StreetLight segments)

def non_traditional_aadt_by_location(
    aadt_locations,
    df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter=None,
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)",
    as_df=True
):
    """
    Build a per-location summary of *non-traditional* (StreetLight) AADT.

    Output columns (one row per location):
      location, daytype_expected, daytype_used, daypart_used, modeoftravel_used,
      ahead_zones, behind_zones,
      non_trad_ahead_mean, non_trad_behind_mean, non_trad_aadt,
      stl_ahead_rows, stl_behind_rows, missing_ahead_zones, missing_behind_zones
    """

    # ---- helpers ----
    def _ensure_list(x):
        if x is None: return []
        if isinstance(x, (list, tuple, set)): return list(x)
        return [x]

    def _gather_zones(node_dict):
        ahead  = _ensure_list(node_dict.get("zonename_ahead", []))
        behind = _ensure_list(node_dict.get("zonename_behind", []))
        return ahead, behind

    def _dedup(seq):
        seen=set(); out=[]
        for x in seq:
            if x not in seen:
                out.append(x); seen.add(x)
        return out

    def _normalize_one_location(name, loc):
        nodes = loc.get("nodes", {}) or {}
        ahead, behind = [], []
        for _, node in nodes.items():
            a, b = _gather_zones(node)
            ahead.extend([z for z in a if z])
            behind.extend([z for z in b if z])
        return {
            "name": name,
            "daytype": loc.get("daytype", "0: All Days (M-Su)"),
            "ahead_zones": _dedup(ahead),
            "behind_zones": _dedup(behind),
        }

    def _normalize_input(aadt_locs):
        # already normalized DataFrame?
        if isinstance(aadt_locs, pd.DataFrame) and \
           {"name","daytype","ahead_zones","behind_zones"}.issubset(aadt_locs.columns):
            return aadt_locs.to_dict(orient="records")
        # already normalized list[dict]?
        if isinstance(aadt_locs, list) and aadt_locs and isinstance(aadt_locs[0], dict) and \
           {"name","daytype","ahead_zones","behind_zones"}.issubset(aadt_locs[0].keys()):
            return aadt_locs

        recs = []
        if isinstance(aadt_locs, dict):
            for nm, loc in aadt_locs.items():
                recs.append(_normalize_one_location(nm, loc))
        elif isinstance(aadt_locs, list):
            for item in aadt_locs:
                if isinstance(item, dict) and "nodes" in item:  # single location dict
                    nm = item.get("location_description") or item.get("name") or "UNKNOWN"
                    recs.append(_normalize_one_location(nm, item))
                elif isinstance(item, dict):  # dict keyed by name
                    for nm, loc in item.items():
                        recs.append(_normalize_one_location(nm, loc))
        return recs

    # ---- filter & precompute per-zone means ----
    must_cols = [zonename_col, stl_volume_col, "daytype", "daypart"]
    for c in must_cols:
        if c not in df_stl.columns:
            raise KeyError(f"df_stl is missing required column: {c}")

    filt = (df_stl["daytype"] == daytype_filter) & (df_stl["daypart"] == daypart_filter)
    if modeoftravel_filter and ("modeoftravel" in df_stl.columns):
        filt = filt & (df_stl["modeoftravel"] == modeoftravel_filter)

    stl_filtered = df_stl.loc[filt, [zonename_col, stl_volume_col]].copy()
    # Clean types
    stl_filtered[zonename_col] = stl_filtered[zonename_col].astype(str).str.strip()
    stl_filtered[stl_volume_col] = pd.to_numeric(stl_filtered[stl_volume_col], errors="coerce")

    # Compute per-zonename averages (handles duplicates safely)
    zone_group = stl_filtered.groupby(zonename_col)[stl_volume_col]
    zone_mean = zone_group.mean()   # pd.Series: index=zonename, value=mean volume
    zone_rows = zone_group.size()   # pd.Series: index=zonename, value=row count backing the mean
    present_zones = set(zone_mean.index)

    def _zone_stats(zones):
        """Return (mean_of_zone_means, backing_row_count_sum, missing_list) for a list of zonenames."""
        zones = [str(z).strip() for z in _ensure_list(zones) if z and str(z).strip() != ""]
        if not zones:
            return np.nan, 0, []
        present = [z for z in zones if z in present_zones]
        missing = [z for z in zones if z not in present_zones]
        means = zone_mean.reindex(present).dropna()
        mean_val = means.mean() if len(means) else np.nan
        # how many filtered rows contributed across these zones (debug)
        n_rows = int(zone_rows.reindex(present).fillna(0).sum())
        return mean_val, n_rows, missing

    # ---- build rows ----
    norm = _normalize_input(aadt_locations)
    rows = []
    for loc in norm:
        ahead = _ensure_list(loc.get("ahead_zones", []))
        behind = _ensure_list(loc.get("behind_zones", []))

        mean_ahead, ahead_n, miss_a = _zone_stats(ahead)
        mean_behind, behind_n, miss_b = _zone_stats(behind)
        overall = np.nanmean([mean_ahead, mean_behind])

        rows.append({
            "location": loc.get("name"),
            "daytype_expected": loc.get("daytype"),
            "daytype_used": daytype_filter,
            "daypart_used": daypart_filter,
            "modeoftravel_used": modeoftravel_filter if modeoftravel_filter else "",
            # Pipe-joined for spreadsheet safety (mirrors Step 1)
            "ahead_zones": "|".join(ahead),
            "behind_zones": "|".join(behind),
            "non_trad_ahead_mean": mean_ahead,
            "non_trad_behind_mean": mean_behind,
            "non_trad_aadt": overall,
            "stl_ahead_rows": ahead_n,
            "stl_behind_rows": behind_n,
            "missing_ahead_zones": "|".join(miss_a) if miss_a else "",
            "missing_behind_zones": "|".join(miss_b) if miss_b else "",
        })

    return pd.DataFrame(rows) if as_df else rows


In [14]:
# this will run the "non_traditional_aadt_by_location" function if  you have the raw nested structure:
stl_df = non_traditional_aadt_by_location(
    sr_99_d3_tc_aadt_locations,
    df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter="All Vehicles - StL All Vehicles Volume",  # or None
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)",
    as_df=True
)



In [15]:
# Export step 2 to a CSV
stl_df.to_csv("step_2_non_traditional_aadt_by_location.csv", index=False)

### Step 3, Build the per-location comparison DataFrame

In [16]:
# # ------------------------------------------------------
# # 3) Build the per-location comparison DataFrame
# # ------------------------------------------------------

def build_aadt_comparison_df(
    aadt_locations,
    df_tc,
    df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter=None,
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)"
) -> pd.DataFrame:
    """
    Build a per-location comparison combining:
      - Traditional (Traffic Census) AADT
      - Non-traditional (StreetLight) AADT
      - TCE (%) = 100 * (non_trad_aadt - traditional_aadt) / traditional_aadt

    Returns a pandas DataFrame (one row per location).
    """

    # 1) Build the two sides using your updated functions
    trad_df = traditional_aadt_by_location(
        aadt_locations=aadt_locations,
        df_tc=df_tc,
        as_df=True
    )

    nt_df = non_traditional_aadt_by_location(
        aadt_locations=aadt_locations,
        df_stl=df_stl,
        daytype_filter=daytype_filter,
        daypart_filter=daypart_filter,
        modeoftravel_filter=modeoftravel_filter,
        zonename_col=zonename_col,
        stl_volume_col=stl_volume_col,
        as_df=True
    )

    # 2) Merge on 'location'
    merged = pd.merge(
        trad_df,
        nt_df,
        how="inner",
        on="location",
        suffixes=("_trad", "_nt")
    )

    # 3) Compute TCE (%), guarding against zero / NaN
    def _tce(row):
        t = row.get("traditional_aadt")
        n = row.get("non_trad_aadt")
        if pd.notna(t) and t != 0 and pd.notna(n):
            return 100.0 * (n - t) / t
        return np.nan

    merged["tce_percent"] = merged.apply(_tce, axis=1)

    # 4) Stable, readable column order (only keep those that exist)
    preferred_cols = [
        "location",
        # IDs & zones (pipe-joined for spreadsheet safety)
        "objectids", "n_objectids", "n_found_in_tc", "missing_objectids",
        "ahead_zones", "behind_zones",
        # AADT metrics
        "traditional_ahead_mean", "traditional_behind_mean", "traditional_aadt",
        "non_trad_ahead_mean", "non_trad_behind_mean", "non_trad_aadt",
        "tce_percent",
        # Filters / metadata
        "daytype",            # from Step 1
        "daytype_expected",   # from Step 2 (original location metadata)
        "daytype_used", "daypart_used", "modeoftravel_used",
        # Debug / row counts
        "stl_ahead_rows", "stl_behind_rows",
        "missing_ahead_zones", "missing_behind_zones",
    ]
    cols = [c for c in preferred_cols if c in merged.columns]
    merged = merged[cols].copy()

    return merged

In [17]:
# 3.1) Build the combined comparison DataFrame
cmp_df = build_aadt_comparison_df(
    aadt_locations=sr_99_d3_tc_aadt_locations,  # your dict/list structure
    df_tc=df_tc,                                 # Traffic Census dataframe
    df_stl=df_stl,                               # StreetLight dataframe
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter=None,                    # e.g., "0: All Modes" if you need it
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)"
)

In [18]:
# 3.2) Quick peek
#cmp_df.head()

In [19]:
# 3.3) (Optional) sort by absolute TCE to see big deltas first
cmp_df = cmp_df.sort_values("tce_percent", key=lambda s: s.abs(), ascending=False)

In [20]:
# 3.4) Export to CSV 
cmp_df.to_csv("step_3_comparison_dataframe.csv", index=False)

## Step 4 Confidence Interval over TCE

In [21]:
# # ------------------------------------------------------
# # 4) Confidence interval over TCE
# # ------------------------------------------------------

def tce_confidence_interval(detail_df, confidence=0.95):
    """
    Compute summary stats over `detail_df["tce_percent"]`.
    Returns: (mean_tce, ci_lo, ci_hi, tcrit, t_stat)
    """
    # Clean and extract
    tces = pd.to_numeric(detail_df["tce_percent"], errors="coerce") \
             .replace([np.inf, -np.inf], np.nan) \
             .dropna().values
    n = len(tces)
    if n == 0:
        return None, None, None, None, None

    mean_tce = float(np.mean(tces))
    if n > 1:
        std_tce = float(np.std(tces, ddof=1))
        se = std_tce / np.sqrt(n)
        if se > 0:
            dof = n - 1
            tcrit = float(stats.t.ppf((1 + confidence) / 2, dof))
            ci_lo = mean_tce - tcrit * se
            ci_hi = mean_tce + tcrit * se
            t_stat = mean_tce / se
        else:
            tcrit = ci_lo = ci_hi = t_stat = None
    else:
        std_tce = 0.0
        se = 0.0
        tcrit = ci_lo = ci_hi = t_stat = None

    return mean_tce, ci_lo, ci_hi, tcrit, t_stat

def tce_confidence_interval_df(detail_df, confidence=0.95) -> pd.DataFrame:
    """
    Same as tce_confidence_interval, but returns a one-row DataFrame with
    extra fields useful for reporting/export.
    """
    tces = pd.to_numeric(detail_df["tce_percent"], errors="coerce") \
             .replace([np.inf, -np.inf], np.nan) \
             .dropna()
    n = int(tces.shape[0])
    if n == 0:
        return pd.DataFrame([{
            "confidence": confidence,
            "n": 0,
            "dof": None,
            "mean_tce": None,
            "std_tce": None,
            "se": None,
            "t_critical": None,
            "margin_of_error": None,
            "ci_lower": None,
            "ci_upper": None,
            "t_statistic": None,
            "p_value_two_sided": None
        }])

    mean_tce = float(tces.mean())
    if n > 1:
        std_tce = float(tces.std(ddof=1))
        se = std_tce / np.sqrt(n)
        dof = n - 1
        if se > 0:
            tcrit = float(stats.t.ppf((1 + confidence) / 2, dof))
            moe = tcrit * se
            ci_lo = mean_tce - moe
            ci_hi = mean_tce + moe
            t_stat = mean_tce / se
            p_val = float(2 * (1 - stats.t.cdf(abs(t_stat), dof)))
        else:
            tcrit = moe = ci_lo = ci_hi = t_stat = p_val = None
    else:
        std_tce = 0.0
        se = 0.0
        dof = None
        tcrit = moe = ci_lo = ci_hi = t_stat = p_val = None

    return pd.DataFrame([{
        "confidence": confidence,
        "n": n,
        "dof": dof,
        "mean_tce": mean_tce,
        "std_tce": std_tce if n > 1 else None,
        "se": se if n > 1 else None,
        "t_critical": tcrit,
        "margin_of_error": moe,
        "ci_lower": ci_lo,
        "ci_upper": ci_hi,
        "t_statistic": t_stat,
        "p_value_two_sided": p_val
    }])


In [22]:
# 4.1) Build your comparison DataFrame first (from prior step)
cmp_df = build_aadt_comparison_df(
    aadt_locations=sr_99_d3_tc_aadt_locations,
    df_tc=df_tc,
    df_stl=df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter=None,
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)"
)

In [23]:
# 4.2) Get the CI summary as a DataFrame
tce_summary_df = tce_confidence_interval_df(cmp_df, confidence=0.95)

In [24]:
# 4.3) Quick peek
print(tce_summary_df)

   confidence   n  dof   mean_tce    std_tce        se  t_critical  \
0        0.95  49   48 -35.124376  27.905644  3.986521    2.010635   

   margin_of_error   ci_lower   ci_upper  t_statistic  p_value_two_sided  
0         8.015437 -43.139813 -27.108939    -8.810785       1.348721e-11  


In [25]:
# 4.4) Export to CSV 
cmp_df.to_csv("step_4_summary.csv", index=False)

In [26]:
mean_tce, ci_lower, ci_upper, t_critical, t_statistic = tce_confidence_interval(
    cmp_df, confidence=0.95
)

print("Mean TCE:", mean_tce)
print("95% Confidence Interval:", (ci_lower, ci_upper))
print("t-test statistic:", t_statistic)
print("t-critical:", t_critical)

Mean TCE: -35.12437585771558
95% Confidence Interval: (-43.13981265550753, -27.10893905992363)
t-test statistic: -8.810785066136184
t-critical: 2.0106347546964454


### Mean TCE: -35.12
Traffic Census Error (TCE)
* Across SR-99 in District 3, StreetLight AADT underestimates Traffic Census AADT by about 35% on average. This appears systematic rather than random, so we’ll (1) fix any mapping/coverage gaps, (2) stratify and weight the results, and (3) apply a corridor-specific calibration before using StreetLight AADT operationally.

### 95% Confidence Interval (-43.14%, -27.11%)
* On SR-99 in District 3, StreetLight AADT underestimates Traffic Census by ~35% on average (95% CI: –43% to –27%), a statistically and operationally significant gap that warrants calibration before use.

### T-Test Statistic (-8.81)
* The corridor-average underestimation is highly significant (t = –8.81, p ≪ 0.001): StreetLight AADT is materially lower than Traffic Census on SR-99 (D3), warranting data QA and corridor-specific calibration.

### t-critical (2.01)
* At 95% confidence, the decision threshold is t-critical = 2.01; our observed t = −8.81 easily exceeds this in magnitude, confirming a statistically significant corridor-level underestimation of StreetLight AADT relative to Traffic Census on SR-99 (D3).

### Summary
* SR-99 (D3): StreetLight AADT is ~35% lower than Traffic Census on average (95% CI –43% to –27%, t = –8.81), indicating a statistically significant corridor-level underestimation that requires QA and calibration before operational use.
